https://github.com/ilguyi/optimizers.numpy

https://ruder.io/optimizing-gradient-descent/index.html#nesterovacceleratedgradient

In this lab, you will implement some of the techniques discussed in the lecture.

Below you are given a solution to the previous scenario. Note that it has two serious drawbacks:
 * The output predictions do not sum up to one (i.e. it does not return a distribution) even though the images always contain exactly one digit.
 * It uses MSE coupled with output sigmoid which can lead to saturation and slow convergence 

**Task 1.** Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence. Hint: When implementing backprop it might be easier to consider these two function as a single block and not even compute the gradient over the softmax values. 

**Task 2.** Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.

**Task 3 (optional).** Implement Adagrad, dropout and some simple data augmentations (e.g. tiny rotations/shifts etc.). Again, test to see how these changes improve accuracy/convergence.

**Task 4.** Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]


In [ ]:
import random
import numpy as np
from torchvision import datasets, transforms

In [ ]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
# standarization
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

wrong_features = (std != 0)[0]

x_train = (x_train - mean) / std #/np.sqrt(x_train.shape[0]) ?
x_test = (x_test - mean) / std

In [ ]:
clip = 5000
def sigmoid(z):
  # z = np.clip(z, -clip, clip)
  return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

def softmax(x):
  max_x = np.max(x, axis=0)[np.newaxis,:]
  exp_x = np.exp(x-max_x)
  return exp_x / exp_x.sum(axis=0)[np.newaxis,:]

def relu(x):
  return np.maximum(x,0)

L1

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.alpha = 0.0001

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        # wagi mniejsze co do wartosci bezwzględnej niż alpha są zerowane
        self.weights = [w-eta*self.alpha*np.sign(np.where(abs(w)>self.alpha,w,0))-(eta/len(x_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        
    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            idx = np.random.permutation(x_train.shape[0])
            x_train = x_train[idx]
            y_train = y_train[idx]
            for i in range(x_train.shape[0] // mini_batch_size):
            # for batch_idx in stratified_split(y_train, mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
                # self.update_mini_batch(x_train[batch_idx], y_train[batch_idx], eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.5, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.8196
Epoch: 1, Accuracy: 0.8601
Epoch: 2, Accuracy: 0.8809
Epoch: 3, Accuracy: 0.8923
Epoch: 4, Accuracy: 0.899
Epoch: 5, Accuracy: 0.9018
Epoch: 6, Accuracy: 0.9056
Epoch: 7, Accuracy: 0.9097
Epoch: 8, Accuracy: 0.9119
Epoch: 9, Accuracy: 0.9146
Epoch: 10, Accuracy: 0.9168
Epoch: 11, Accuracy: 0.9193
Epoch: 12, Accuracy: 0.9207
Epoch: 13, Accuracy: 0.9233
Epoch: 14, Accuracy: 0.9232
Epoch: 15, Accuracy: 0.9249
Epoch: 16, Accuracy: 0.9248
Epoch: 17, Accuracy: 0.9281
Epoch: 18, Accuracy: 0.9277
Epoch: 19, Accuracy: 0.9281
Epoch: 20, Accuracy: 0.9317
Epoch: 21, Accuracy: 0.9312
Epoch: 22, Accuracy: 0.9314
Epoch: 23, Accuracy: 0.9332
Epoch: 24, Accuracy: 0.9321
Epoch: 25, Accuracy: 0.9348
Epoch: 26, Accuracy: 0.935
Epoch: 27, Accuracy: 0.934
Epoch: 28, Accuracy: 0.9348
Epoch: 29, Accuracy: 0.9365
Epoch: 30, Accuracy: 0.9366
Epoch: 31, Accuracy: 0.9361
Epoch: 32, Accuracy: 0.9368
Epoch: 33, Accuracy: 0.9395
Epoch: 34, Accuracy: 0.9377
Epoch: 35, Accuracy: 0.9395
Epoch

Adagrad

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.m_b = [np.zeros_like(w) for w in self.biases]
        self.m_w = [np.zeros_like(w) for w in self.weights]
        self.eps = 1e-8

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)

        self.m_b = [m + (nb/len(x_batch))**2 for m, nb in zip(self.m_b, nabla_b)]
        self.biases = [b - (eta / np.sqrt(m + self.eps)) * nb / len(x_batch)\
                       for b, nb, m in zip(self.biases, nabla_b, self.m_b)]

        self.m_w = [m + (nb/len(x_batch))**2 for m, nb in zip(self.m_w, nabla_w)]
        self.weights = [w - (eta/np.sqrt(m + self.eps)) * nw / len(x_batch)\
                        for w, nw, m in zip(self.weights, nabla_w, self.m_w)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.1, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.9008
Epoch: 1, Accuracy: 0.914
Epoch: 2, Accuracy: 0.9198
Epoch: 3, Accuracy: 0.9248
Epoch: 4, Accuracy: 0.9295
Epoch: 5, Accuracy: 0.9314
Epoch: 6, Accuracy: 0.9334
Epoch: 7, Accuracy: 0.9358
Epoch: 8, Accuracy: 0.9367
Epoch: 9, Accuracy: 0.9376
Epoch: 10, Accuracy: 0.9393
Epoch: 11, Accuracy: 0.94
Epoch: 12, Accuracy: 0.9411
Epoch: 13, Accuracy: 0.9417
Epoch: 14, Accuracy: 0.9425
Epoch: 15, Accuracy: 0.9435
Epoch: 16, Accuracy: 0.9442
Epoch: 17, Accuracy: 0.9452
Epoch: 18, Accuracy: 0.9458
Epoch: 19, Accuracy: 0.9457
Epoch: 20, Accuracy: 0.9459
Epoch: 21, Accuracy: 0.9464
Epoch: 22, Accuracy: 0.9472
Epoch: 23, Accuracy: 0.9474
Epoch: 24, Accuracy: 0.9477
Epoch: 25, Accuracy: 0.9475
Epoch: 26, Accuracy: 0.9476
Epoch: 27, Accuracy: 0.9476
Epoch: 28, Accuracy: 0.9478
Epoch: 29, Accuracy: 0.9475
Epoch: 30, Accuracy: 0.9478
Epoch: 31, Accuracy: 0.9479
Epoch: 32, Accuracy: 0.9482
Epoch: 33, Accuracy: 0.9484
Epoch: 34, Accuracy: 0.9487
Epoch: 35, Accuracy: 0.9488
Epoch

Nesterov's Momentum

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.m_b = [np.zeros_like(b) for b in self.biases]
        self.m_w = [np.zeros_like(w) for w in self.weights]
        self.mu = 0.9

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate

        # PROPER WAY?
        # for i in range(len(self.biases)):
        #   w_org, b_org = self.weights[i].copy(), self.biases[i].copy()
        #   self.weights[i] -= self.mu*self.m_w[i]
        #   self.biases[i] -= self.mu*self.m_b[i]
        #   nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        #   self.m_w[i] = self.mu*self.m_w[i] + (eta/len(x_batch))*nabla_w[i]
        #   self.m_b[i] = self.mu*self.m_b[i] + (eta/len(x_batch))*nabla_b[i]
        #   self.weights[i], self.biases[i] = w_org, b_org

        self.weights = [w - self.mu*m for w, m in zip(self.weights, self.m_w)]
        self.biases = [b - self.mu*m for b, m in zip(self.biases, self.m_b)]
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        self.weights = [w + self.mu*m for w, m in zip(self.weights, self.m_w)]
        self.biases = [b + self.mu*m for b, m in zip(self.biases, self.m_b)]

        self.m_w = [self.mu*m + (eta/len(x_batch))*nw for m, nw in zip(self.m_w, nabla_w)]
        self.m_b = [self.mu*m + (eta/len(x_batch))*nb for m, nb in zip(self.m_b, nabla_b)]

        self.weights = [w - g for w, g in zip(self.weights, self.m_w)]
        self.biases = [b - g for b, g in zip(self.biases, self.m_b)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.1, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.8711
Epoch: 1, Accuracy: 0.8973
Epoch: 2, Accuracy: 0.9094
Epoch: 3, Accuracy: 0.9157
Epoch: 4, Accuracy: 0.9211
Epoch: 5, Accuracy: 0.9262
Epoch: 6, Accuracy: 0.9294
Epoch: 7, Accuracy: 0.9318
Epoch: 8, Accuracy: 0.9331
Epoch: 9, Accuracy: 0.9336
Epoch: 10, Accuracy: 0.9354
Epoch: 11, Accuracy: 0.9361
Epoch: 12, Accuracy: 0.939
Epoch: 13, Accuracy: 0.9401
Epoch: 14, Accuracy: 0.9403
Epoch: 15, Accuracy: 0.9409
Epoch: 16, Accuracy: 0.9408
Epoch: 17, Accuracy: 0.941
Epoch: 18, Accuracy: 0.9413
Epoch: 19, Accuracy: 0.9415
Epoch: 20, Accuracy: 0.9428
Epoch: 21, Accuracy: 0.9438
Epoch: 22, Accuracy: 0.9443
Epoch: 23, Accuracy: 0.9451
Epoch: 24, Accuracy: 0.9454
Epoch: 25, Accuracy: 0.9461
Epoch: 26, Accuracy: 0.9467
Epoch: 27, Accuracy: 0.9467
Epoch: 28, Accuracy: 0.9467
Epoch: 29, Accuracy: 0.947
Epoch: 30, Accuracy: 0.9473
Epoch: 31, Accuracy: 0.9474
Epoch: 32, Accuracy: 0.9475
Epoch: 33, Accuracy: 0.9473
Epoch: 34, Accuracy: 0.9478
Epoch: 35, Accuracy: 0.9479
Epoch

RMSProp

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.m_b = [np.zeros_like(w) for w in self.biases]
        self.m_w = [np.zeros_like(w) for w in self.weights]
        self.mu = 0.9
        self.eps = 1e-8

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        
        self.m_b = [self.mu * m + (1 - self.mu) * (nb/len(x_batch))**2 \
                    for m, nb in zip(self.m_b, nabla_b)]
        self.biases = [b - (eta / np.sqrt(m + self.eps)) * (nb/len(x_batch)) \
                       for b, nb, m in zip(self.biases, nabla_b, self.m_b)]
        
        self.m_w = [self.mu * m + (1 - self.mu) * (nw/len(x_batch))**2 \
                    for m, nw in zip(self.m_w, nabla_w)]
        self.weights = [b - (eta/np.sqrt(m + self.eps)) * (nw/len(x_batch)) \
                        for b, nw, m in zip(self.weights, nabla_w, self.m_w)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.01, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.9045
Epoch: 1, Accuracy: 0.9196
Epoch: 2, Accuracy: 0.9298
Epoch: 3, Accuracy: 0.9345
Epoch: 4, Accuracy: 0.938
Epoch: 5, Accuracy: 0.9418
Epoch: 6, Accuracy: 0.9454
Epoch: 7, Accuracy: 0.9465
Epoch: 8, Accuracy: 0.9446
Epoch: 9, Accuracy: 0.9466
Epoch: 10, Accuracy: 0.9477
Epoch: 11, Accuracy: 0.9467
Epoch: 12, Accuracy: 0.9472
Epoch: 13, Accuracy: 0.9462
Epoch: 14, Accuracy: 0.9452
Epoch: 15, Accuracy: 0.9461
Epoch: 16, Accuracy: 0.9462
Epoch: 17, Accuracy: 0.9454
Epoch: 18, Accuracy: 0.9468
Epoch: 19, Accuracy: 0.947
Epoch: 20, Accuracy: 0.9463
Epoch: 21, Accuracy: 0.947
Epoch: 22, Accuracy: 0.9476
Epoch: 23, Accuracy: 0.9477
Epoch: 24, Accuracy: 0.946
Epoch: 25, Accuracy: 0.9464
Epoch: 26, Accuracy: 0.9479
Epoch: 27, Accuracy: 0.95
Epoch: 28, Accuracy: 0.9495
Epoch: 29, Accuracy: 0.9501
Epoch: 30, Accuracy: 0.9488
Epoch: 31, Accuracy: 0.9485
Epoch: 32, Accuracy: 0.9475
Epoch: 33, Accuracy: 0.9472
Epoch: 34, Accuracy: 0.9484
Epoch: 35, Accuracy: 0.948
Epoch: 36

Adam

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.m_w = [np.zeros_like(w) for w in self.weights]
        self.g_w = [np.zeros_like(w) for w in self.weights]
        self.m_b = [np.zeros_like(w) for w in self.biases]
        self.g_b = [np.zeros_like(w) for w in self.biases]
        self.beta2 = 0.999
        self.beta1 = 0.9
        self.eps = 1e-8

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta, epoch):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)

        self.m_w = [self.beta1 * m + (1 - self.beta1) * (nw/len(x_batch))\
                  for m, nw in zip(self.m_w, nabla_w)]
        self.g_w = [self.beta2 * g + (1 - self.beta2) * (nw/len(x_batch))**2\
                  for g, nw in zip(self.g_w, nabla_w)]
        self.weights = [w - (eta / (np.sqrt(g/(1 - self.beta2 ** epoch)) + self.eps)) * m/(1 - self.beta1 ** epoch)\
                        for w, m, g in zip(self.weights, self.m_w, self.g_w)]

        self.m_b = [self.beta1 * m + (1 - self.beta1) * (nw/len(x_batch))\
                  for m, nw in zip(self.m_b, nabla_b)]
        self.g_b = [self.beta2 * g + (1 - self.beta2) * (nw/len(x_batch))**2\
                  for g, nw in zip(self.g_b, nabla_b)]
        self.biases = [w - (eta / (np.sqrt(g/(1 - self.beta2 ** epoch)) + self.eps)) * m/(1 - self.beta1 ** epoch)\
                        for w, m, g in zip(self.biases, self.m_b, self.g_b)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            idx = np.random.permutation(x_train.shape[0])
            x_train = x_train[idx]
            y_train = y_train[idx]
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta, j+1)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.1, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.9198
Epoch: 1, Accuracy: 0.9325
Epoch: 2, Accuracy: 0.9373
Epoch: 3, Accuracy: 0.9409
Epoch: 4, Accuracy: 0.9424
Epoch: 5, Accuracy: 0.9457
Epoch: 6, Accuracy: 0.9468
Epoch: 7, Accuracy: 0.944
Epoch: 8, Accuracy: 0.9484
Epoch: 9, Accuracy: 0.9469
Epoch: 10, Accuracy: 0.9487
Epoch: 11, Accuracy: 0.9503
Epoch: 12, Accuracy: 0.9498
Epoch: 13, Accuracy: 0.9482
Epoch: 14, Accuracy: 0.9515
Epoch: 15, Accuracy: 0.9503
Epoch: 16, Accuracy: 0.9501
Epoch: 17, Accuracy: 0.9493
Epoch: 18, Accuracy: 0.9516
Epoch: 19, Accuracy: 0.9493
Epoch: 20, Accuracy: 0.9473
Epoch: 21, Accuracy: 0.9521
Epoch: 22, Accuracy: 0.9533
Epoch: 23, Accuracy: 0.9494
Epoch: 24, Accuracy: 0.9501
Epoch: 25, Accuracy: 0.9514
Epoch: 26, Accuracy: 0.9502
Epoch: 27, Accuracy: 0.9508
Epoch: 28, Accuracy: 0.9516
Epoch: 29, Accuracy: 0.9484
Epoch: 30, Accuracy: 0.9499
Epoch: 31, Accuracy: 0.9509
Epoch: 32, Accuracy: 0.9524
Epoch: 33, Accuracy: 0.9533
Epoch: 34, Accuracy: 0.9498
Epoch: 35, Accuracy: 0.9526
Epo

Dropout

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        self.weights = [w-(eta/len(x_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        dropout_mask = np.concatenate([np.random])
        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=1.0, test_data=(x_test, y_test))



Augmentation

Swap adjacent rows and cols

In [ ]:
import torch

In [ ]:
torch.distributions.Normal(0, torch.tensor([1.0]))

In [ ]:
def aug(x, p):
  for el in x:
    if np.random.rand() < p:
      nb = np.random.randint(0, el.shape[0]-1)
      el[[nb,nb+1]] = el[[nb+1, nb]]
  return x

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(aug(x_batch.copy(), 0.3).T, y_batch.T)
        self.weights = [w-(eta/len(x_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=1.0, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.8553
Epoch: 1, Accuracy: 0.885
Epoch: 2, Accuracy: 0.8998
Epoch: 3, Accuracy: 0.9073
Epoch: 4, Accuracy: 0.9115
Epoch: 5, Accuracy: 0.9152
Epoch: 6, Accuracy: 0.9187
Epoch: 7, Accuracy: 0.9219
Epoch: 8, Accuracy: 0.9238
Epoch: 9, Accuracy: 0.9257
Epoch: 10, Accuracy: 0.9284
Epoch: 11, Accuracy: 0.9305
Epoch: 12, Accuracy: 0.9323
Epoch: 13, Accuracy: 0.9341
Epoch: 14, Accuracy: 0.9352
Epoch: 15, Accuracy: 0.9358
Epoch: 16, Accuracy: 0.9368
Epoch: 17, Accuracy: 0.9371
Epoch: 18, Accuracy: 0.9386
Epoch: 19, Accuracy: 0.9391
Epoch: 20, Accuracy: 0.9392
Epoch: 21, Accuracy: 0.9397
Epoch: 22, Accuracy: 0.9405
Epoch: 23, Accuracy: 0.941
Epoch: 24, Accuracy: 0.9408
Epoch: 25, Accuracy: 0.9414
Epoch: 26, Accuracy: 0.9415
Epoch: 27, Accuracy: 0.9422
Epoch: 28, Accuracy: 0.9428
Epoch: 29, Accuracy: 0.9427
Epoch: 30, Accuracy: 0.9435
Epoch: 31, Accuracy: 0.9431
Epoch: 32, Accuracy: 0.9436
Epoch: 33, Accuracy: 0.9432
Epoch: 34, Accuracy: 0.944
Epoch: 35, Accuracy: 0.9437
Epoch

Softmax

In [ ]:
def stratified_split(y_train, bs):
  n_class = np.unique(y_train).shape[0]
  splitted_y = [(y_train == i).nonzero()[0] for i in np.unique(y_train)]
  for _ in range(y_train.shape[0] // bs):
    batch_idx = []
    for i in range(n_class):
      sample = splitted_y[i][:bs // n_class]
      splitted_y[i] = np.delete(splitted_y[i], np.where(splitted_y[i] == sample))
      batch_idx += sample.tolist()
    yield np.random.permutation(np.array(batch_idx))
  

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return h
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        self.weights = [w-(eta/len(x_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(h, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (softmax(output_activations)-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            idx = np.random.permutation(x_train.shape[0])
            x_train = x_train[idx]
            y_train = y_train[idx]
            for i in range(x_train.shape[0] // mini_batch_size):
            # for batch_idx in stratified_split(y_train, mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
                # self.update_mini_batch(x_train[batch_idx], y_train[batch_idx], eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=1.0, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.8602
Epoch: 1, Accuracy: 0.8876
Epoch: 2, Accuracy: 0.9001
Epoch: 3, Accuracy: 0.9094
Epoch: 4, Accuracy: 0.915
Epoch: 5, Accuracy: 0.9175
Epoch: 6, Accuracy: 0.9228
Epoch: 7, Accuracy: 0.9247
Epoch: 8, Accuracy: 0.927
Epoch: 9, Accuracy: 0.9308
Epoch: 10, Accuracy: 0.9322
Epoch: 11, Accuracy: 0.9347
Epoch: 12, Accuracy: 0.934
Epoch: 13, Accuracy: 0.9353
Epoch: 14, Accuracy: 0.9367
Epoch: 15, Accuracy: 0.9399
Epoch: 16, Accuracy: 0.9405
Epoch: 17, Accuracy: 0.9417
Epoch: 18, Accuracy: 0.9407
Epoch: 19, Accuracy: 0.9436
Epoch: 20, Accuracy: 0.9424
Epoch: 21, Accuracy: 0.9427
Epoch: 22, Accuracy: 0.9434
Epoch: 23, Accuracy: 0.9433
Epoch: 24, Accuracy: 0.9447
Epoch: 25, Accuracy: 0.9457
Epoch: 26, Accuracy: 0.9458
Epoch: 27, Accuracy: 0.9468
Epoch: 28, Accuracy: 0.9468
Epoch: 29, Accuracy: 0.9462
Epoch: 30, Accuracy: 0.9469
Epoch: 31, Accuracy: 0.9477
Epoch: 32, Accuracy: 0.9465
Epoch: 33, Accuracy: 0.9489
Epoch: 34, Accuracy: 0.9472
Epoch: 35, Accuracy: 0.9492
Epoch

In [ ]:
np.random.permutation(np.arange(2,8))

array([3, 7, 2, 6, 4, 5])

In [ ]:
y_train.shape

(60000, 10)

Softmax + ReLU

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = relu(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        return softmax(h)
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        self.weights = [w-(eta/len(x_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h1 = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = relu(h1)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        a2 = softmax(h)
        
        # Now go backward from the final cost applying backpropagation
        ph2 = self.cost_derivative(a2, y_batch)
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T
        
        ph1 = self.weights[1].T @ ph2
        ph1[h1 < 0] = 0
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.5, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.8175
Epoch: 1, Accuracy: 0.8564
Epoch: 2, Accuracy: 0.8727
Epoch: 3, Accuracy: 0.8852
Epoch: 4, Accuracy: 0.8885
Epoch: 5, Accuracy: 0.8936
Epoch: 6, Accuracy: 0.8983
Epoch: 7, Accuracy: 0.9027
Epoch: 8, Accuracy: 0.9071
Epoch: 9, Accuracy: 0.9135
Epoch: 10, Accuracy: 0.9154
Epoch: 11, Accuracy: 0.9178
Epoch: 12, Accuracy: 0.9187


KeyboardInterrupt: ignored

Weight Decay

In [ ]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.alpha = 0.0001

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        h = np.dot(self.weights[0], a) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        a2 = sigmoid(h)
        h = np.dot(self.weights[2], a2) + self.biases[2]
        return softmax(h)
    
    def update_mini_batch(self, x_batch, y_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b, nabla_w = self.backprop(x_batch.T, y_batch.T)
        self.weights = [w-(eta/len(x_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(x_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    

    def backprop(self, x_batch, y_batch):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.
        
        # First initialize the list of gradient arrays
        delta_nabla_b = [np.zeros_like(p) for p in self.biases]
        delta_nabla_w = [np.zeros_like(p) for p in self.weights]

        h = np.dot(self.weights[0], x_batch) + self.biases[0]
        a1 = sigmoid(h)
        h = np.dot(self.weights[1], a1) + self.biases[1]
        a2 = sigmoid(h)
        # a1 = relu(h)
        h = np.dot(self.weights[2], a2) + self.biases[2]
        a3 = softmax(h)
        
        # Now go backward from the final cost applying backpropagation
        ph3 = self.cost_derivative(a3, y_batch)
        delta_nabla_b[2] += ph3.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[2] += ph3 @ a2.T
        
        ph2 = self.weights[2].T @ ph3 * a2 * (1 - a2)
        # ph1[a1 < 0] = 0
        delta_nabla_b[1] += ph2.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[1] += ph2 @ a1.T

        ph1 = self.weights[1].T @ ph2 * a1 * (1 - a1)
        # ph1[a1 < 0] = 0
        delta_nabla_b[0] += ph1.sum(axis=1)[:, np.newaxis]
        delta_nabla_w[0] += ph1 @ x_batch.T

        return (delta_nabla_b, delta_nabla_w)
        

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,100,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.5, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.8231
Epoch: 1, Accuracy: 0.8727
Epoch: 2, Accuracy: 0.8913
Epoch: 3, Accuracy: 0.9027
Epoch: 4, Accuracy: 0.9094
Epoch: 5, Accuracy: 0.9156
Epoch: 6, Accuracy: 0.92
Epoch: 7, Accuracy: 0.9239
Epoch: 8, Accuracy: 0.9266
Epoch: 9, Accuracy: 0.929
Epoch: 10, Accuracy: 0.931
Epoch: 11, Accuracy: 0.9324
Epoch: 12, Accuracy: 0.9338
Epoch: 13, Accuracy: 0.9346
Epoch: 14, Accuracy: 0.936
Epoch: 15, Accuracy: 0.9372
Epoch: 16, Accuracy: 0.9383
Epoch: 17, Accuracy: 0.9389
Epoch: 18, Accuracy: 0.9393
Epoch: 19, Accuracy: 0.94
Epoch: 20, Accuracy: 0.9407
Epoch: 21, Accuracy: 0.9418
Epoch: 22, Accuracy: 0.9419
Epoch: 23, Accuracy: 0.9429
Epoch: 24, Accuracy: 0.9435
Epoch: 25, Accuracy: 0.9441
Epoch: 26, Accuracy: 0.9445
Epoch: 27, Accuracy: 0.9445
Epoch: 28, Accuracy: 0.9446
Epoch: 29, Accuracy: 0.9447
Epoch: 30, Accuracy: 0.9442
Epoch: 31, Accuracy: 0.9445
Epoch: 32, Accuracy: 0.9444
Epoch: 33, Accuracy: 0.9447
Epoch: 34, Accuracy: 0.9447
Epoch: 35, Accuracy: 0.9448
Epoch: 36

Weight Decay + Momentum

Inne wersje backprop

In [ ]:
    # def backprop(self, x, y):
    #     # For a single input (x,y) return a pair of lists.
    #     # First contains gradients over biases, second over weights.
    #     g = x
    #     gs = [g] # list to store all the gs, layer by layer
    #     fs = [] # list to store all the fs, layer by layer
    #     for b, w in zip(self.biases, self.weights):
    #         f = np.dot(w, g)+b
    #         fs.append(f)
    #         g = sigmoid(f)
    #         gs.append(g)
    #     # backward pass <- both steps at once
    #     dLdg = self.cost_derivative(gs[-1], y)
    #     dLdfs = []
    #     for w,g in reversed(list(zip(self.weights,gs[1:]))):
    #         dLdf = np.multiply(dLdg,np.multiply(g,1-g))
    #         dLdfs.append(dLdf)
    #         dLdg = np.matmul(w.T, dLdf)
        
    #     dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] 
    #     dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 
    #     return (dLdBs,dLdWs)


    # def backprop(self, x_batch, y_batch):
    #     # For a single input (x,y) return a tuple of lists.
    #     # First contains gradients over biases, second over weights.
        
    #     # First initialize the list of gradient arrays
    #     delta_nabla_b = [np.zeros_like(p) for p in self.biases]
    #     delta_nabla_w = [np.zeros_like(p) for p in self.weights]
        
    #     # Then go forward remembering all values before and after activations
    #     # in two other array lists
    #     a = x_batch
    #     post_act = []
    #     for w, b in zip(self.weights, self.biases):
    #       a = sigmoid(np.dot(w, a) + b)
    #       post_act.append(a)
        
    #     # Now go backward from the final cost applying backpropagation
    #     ph2 = np.multiply((post_act[1] - y_batch), np.multiply(post_act[1], (1 - post_act[1])))
    #     delta_nabla_b[1] += np.sum(ph2,axis=1).reshape(ph2.shape[0],1)#ph2.sum(axis=1)[:, np.newaxis]
    #     delta_nabla_w[1] += np.matmul(ph2, post_act[0].T) #+ 0.0001 * x_batch.shape[1] * self.weights[1]
        
    #     ph1 = np.multiply(np.matmul(self.weights[1].T, ph2), np.multiply(post_act[0], (1 - post_act[0])))
    #     delta_nabla_b[0] += np.sum(ph1,axis=1).reshape(ph1.shape[0],1)#ph1.sum(axis=1)[:, np.newaxis]
    #     delta_nabla_w[0] += np.matmul(ph1, x_batch.T) #+ 0.0001 * x_batch.shape[1] * self.weights[0]

    #     return (delta_nabla_b, delta_nabla_w)